# 📌 Extracción

In [1]:
import pandas as pd
import json

# Cargar el JSON local (ajusta la ruta si es necesario)
with open("TelecomX_Data.json", "r") as f:
    data = json.load(f)

# Convertir a DataFrame
df = pd.DataFrame(data)
print("Primeras filas de los datos extraídos:")
display(df.head())
print("Dimensiones del dataset:", df.shape)

Primeras filas de los datos extraídos:


,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


Dimensiones del dataset: (7267, 6)


# 🔧 Transformación

In [ ]:
df = df.drop_duplicates()

print("Valores nulos por columna:")
print(df.isnull().sum())

df = df.dropna()

cat_cols = ["gender", "Partner", "Dependents", "PhoneService", "InternetService",
            "Contract", "PaymentMethod", "Churn"]

for col in cat_cols:
    if col in df.columns:
        df[col] = df[col].astype("category")

# Convertir columnas numéricas (si existen en los datos)
num_cols = ["tenure", "MonthlyCharges", "TotalCharges"]
for col in num_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Revisar después de la transformación
df.info()

# 📊 Carga y análisis

In [5]:
def expand_customer(df):
    """Expande la columna customer en nuevas columnas dentro del mismo DataFrame"""
    customer_df = pd.json_normalize(df["customer"])
    df = df.drop(columns=["customer"])
    return pd.concat([df, customer_df], axis=1)

def expand_phone(df):
    """Expande la columna phone"""
    phone_df = pd.json_normalize(df["phone"])
    df = df.drop(columns=["phone"])
    return pd.concat([df, phone_df], axis=1)

def expand_internet(df):
    """Expande la columna internet"""
    internet_df = pd.json_normalize(df["internet"])
    df = df.drop(columns=["internet"])
    return pd.concat([df, internet_df], axis=1)

def expand_account(df):
    """Expande la columna account y sus subniveles"""
    account_df = pd.json_normalize(df["account"])
    
    # renombrar Charges.Monthly → MonthlyCharges, Charges.Total → TotalCharges
    account_df = account_df.rename(columns={
        "Charges.Monthly": "MonthlyCharges",
        "Charges.Total": "TotalCharges"
    })
    
    df = df.drop(columns=["account"])
    return pd.concat([df, account_df], axis=1)


def clean_telecom_data(df):
    df = expand_customer(df)
    df = expand_phone(df)
    df = expand_internet(df)
    df = expand_account(df)
    
    # Convertir TotalCharges a numérico
    if "TotalCharges" in df.columns:
        df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")
    
    # Ajustar tipos de datos
    cat_cols = ["gender", "Partner", "Dependents", "PhoneService", "MultipleLines",
                "InternetService", "OnlineSecurity", "OnlineBackup", "DeviceProtection",
                "TechSupport", "StreamingTV", "StreamingMovies", "Contract",
                "PaperlessBilling", "PaymentMethod"]
    
    for col in cat_cols:
        if col in df.columns:
            df[col] = df[col].astype("category")
    
    num_cols = ["SeniorCitizen", "tenure", "MonthlyCharges", "TotalCharges"]
    for col in num_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")
    
    return df

In [7]:
# Limpiar / expandir
df_clean = clean_telecom_data(df)

print(df_clean.head())

   customerID Churn  gender  SeniorCitizen Partner Dependents  tenure  \
0  0002-ORFBO    No  Female              0     Yes        Yes       9   
1  0003-MKNFE    No    Male              0      No         No       9   
2  0004-TLHLJ   Yes    Male              0      No         No       4   
3  0011-IGKFF   Yes    Male              1     Yes         No      13   
4  0013-EXCHZ   Yes  Female              1     Yes         No       3   

  PhoneService MultipleLines InternetService  ... OnlineBackup  \
0          Yes            No             DSL  ...          Yes   
1          Yes           Yes             DSL  ...           No   
2          Yes            No     Fiber optic  ...           No   
3          Yes            No     Fiber optic  ...          Yes   
4          Yes            No     Fiber optic  ...           No   

  DeviceProtection TechSupport StreamingTV StreamingMovies        Contract  \
0               No         Yes         Yes              No        One year   
1       

In [9]:
#imprime el nombre de todas las cols
print(df_clean.columns.tolist())

['customerID', 'Churn', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges']


# 📄Informe final

## 🎯 Objetivo
El objetivo de este análisis es **identificar los factores que influyen en la cancelación del servicio (Churn)** de clientes en una empresa de telecomunicaciones. Comprender estas variables permite diseñar estrategias para **reducir la tasa de abandono**, mejorar la retención de clientes y orientar decisiones de negocio.

---

## 📂 Datos Utilizados
El dataset final contiene las siguientes variables:

- **Identificación del cliente:**  
  `customerID`

- **Variable objetivo:**  
  `Churn` (Sí/No: indica si el cliente canceló el servicio)

- **Datos demográficos:**  
  `gender`, `SeniorCitizen`, `Partner`, `Dependents`

- **Información de servicio:**  
  `tenure` (meses con la compañía),  
  `PhoneService`, `MultipleLines`,  
  `InternetService`, `OnlineSecurity`, `OnlineBackup`, `DeviceProtection`, `TechSupport`,  
  `StreamingTV`, `StreamingMovies`

- **Información contractual y de facturación:**  
  `Contract` (tipo de contrato),  
  `PaperlessBilling`,  
  `PaymentMethod`,  
  `MonthlyCharges`,  
  `TotalCharges`

---

## 📌 Conclusiones
- El **Churn está influenciado principalmente por la modalidad de contrato, el tipo de servicio de internet, la antigüedad del cliente y los cargos mensuales.**  
- Estrategias para retener clientes:  
  - Incentivar contratos a largo plazo con beneficios atractivos.  
  - Fortalecer servicios de soporte y seguridad.  
  - Revisar esquemas de precios para clientes con cargos elevados.  
  - Enfocar programas de fidelización en clientes con baja permanencia.

---

## 🚀 Próximos Pasos
- Construcción de un **EDA de Churn** utilizando las variables limpias.  
- Evaluación de la importancia de cada variable mediante técnicas de machine learning.  
- Propuesta de estrategias de retención basadas en segmentación de clientes de alto riesgo.

---